In [1]:
import numpy as np
import pandas as pd
import pickle
from nltk.tokenize import TreebankWordTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2023-04-20 12:41:43.950552: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
review = []
for i in range(1, 8):
    review.append(pd.read_csv(f'data/review{i}.csv'))
review = pd.concat(review, ignore_index=True)
review

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
1,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1,1,2,1,I am a long term frequent customer of this est...,2015-09-23 23:10:31
2,8JFGBuHMoiNDyfcxuWNtrA,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4,0,0,0,Good food--loved the gnocchi with marinara\nth...,2009-10-14 19:57:14
3,oyaMhzBSwfGgemSGuZCdwQ,Dd1jQj7S-BFGqRbApFzCFw,YtSqYv1Q_pOltsVPSx54SA,5,0,0,0,Tremendous service (Big shout out to Douglas) ...,2013-06-24 11:21:25
4,Xs8Z8lmKkosqW5mw_sVAoA,IQsF3Rc6IgCzjVV9DE8KXg,eFvzHawVJofxSnD7TgbZtg,5,0,0,0,My absolute favorite cafe in the city. Their b...,2014-11-12 15:30:27
...,...,...,...,...,...,...,...,...,...
687284,Wrt6pZizQzw-ZTKrvMwrGw,ua6QuBe6mar6pDrhHETzJQ,Bk_1vsPtOtO0bojfQZQIOw,2,3,3,2,"Ok, I have to say, after living on this block ...",2008-12-15 23:18:27
687285,X5R98ygOtbhryDiKA-J2qQ,LHWtjTG7e1NzNPYUbUo-9w,rgeuy1qbw6Z8B6CSVANHIA,5,1,1,1,I've been to the other Federal Donuts location...,2012-10-13 14:39:37
687286,MVg4YUQeEhCA7Z7RsBJSVg,7-7A0Avj47slLGV7yBFc8w,ytynqOUb3hjKeJfRj5Tshw,3,1,0,0,"I was so excited about all the food I saw, but...",2013-07-25 21:00:15
687287,nLjbVsETpqO17RbFcqskkA,am7-gkH_PDz598oTdYSD6A,3gVSrS4kffGGZT8oXHsIcw,3,2,0,2,"*Later Yelp* I've only been here once, but I l...",2014-11-03 14:45:46


In [3]:
def clean(string):
    return TreebankWordTokenizer().tokenize(string.lower())

review['text'] = review['text'].apply(clean)
words = review['text'].apply(set)
words = set().union(*words)
len(words)

365243

In [4]:
word2idx = {w: i + 1 for i, w in enumerate(words)}
f = open("data/word2idx.pkl", "wb")
pickle.dump(word2idx, f)
f.close()

In [5]:
X = review['text'].apply(lambda r: [word2idx[w] for w in r])
X

0         [254929, 79360, 55682, 319888, 66617, 319888, ...
1         [318324, 358926, 211940, 198440, 340779, 29641...
2         [309650, 225259, 114253, 232146, 32407, 190571...
3         [290475, 324746, 30129, 313303, 338737, 326985...
4         [104641, 88524, 35451, 63094, 255688, 32407, 5...
                                ...                        
687284    [71497, 319888, 318324, 171514, 143085, 54122,...
687285    [318324, 207181, 40781, 143085, 32407, 99467, ...
687286    [318324, 162676, 350605, 280013, 362460, 18563...
687287    [90095, 82718, 318324, 207181, 26702, 40781, 1...
687288    [288024, 187705, 211940, 309650, 138984, 14385...
Name: text, Length: 687289, dtype: object

In [6]:
max_words = max([len(i) for i in X])
X = pad_sequences(X, max_words)
X.shape

(687289, 1093)

In [7]:
y = review['stars'].to_numpy()
np.savez_compressed('data/reviews.npz', X=X, y=y)